In [ ]:
#%%
import sys
sys.path.append('../')
from pdtb2 import CorpusReader, Datum
import matplotlib
matplotlib.use('Agg')
#%% md
learn more things from [github](https://github.com/cgpotts/pdtb2)
#%%
iterator = CorpusReader('../pdtb2.csv').iter_data(display_progress=False)
#%%
for _ in range(17): 
    next(iterator)
#%%
d = next(iterator)
#%%
d.arg1_attribution_pos()
#%%
len(CorpusReader('../pdtb2.csv'))
#%%
next(iterator)
d = next(iterator)
#%%
d.Relation, d.Section, d.FileNumber, d.Connective_Trees
#%%
d.FullRawText
#%%
d.Arg2_Trees
#%%
[list(o) for o in d.Arg1_GornList]
#%% md
## prepare dataset
#%%
import pandas as pd
import spacy,en_core_web_sm
from stanfordcorenlp import StanfordCoreNLP
import re
#%%
pdtb2 = pd.read_csv('../pdtb2.csv', low_memory=False)
#%%
pdtb2.ix[4000, 'Section']
#%%
pdtb2.head()
#%% md
### file number
#%%
print("file number is: " + f"{pdtb2.ix[4, 'Section']}_{pdtb2.ix[4, 'FileNumber']}")
#%% md
### imports
#%%

In [ ]:
import pandas as pd
import spacy,en_core_web_sm
from stanfordcorenlp import StanfordCoreNLP
import re
from pathlib import Path
import json
tokenizor = en_core_web_sm.load()
nlp = StanfordCoreNLP(r'/home/pengfei/documents/stanford-corenlp-full-2018-10-05/')
sentence = "On Tuesday afternoon , Kemper told Bear , Stearns & Co. , General Electric Co. 's Kidder , Peabody & Co. unit , Morgan Stanley and Oppenheimer & Co. that it'll no longer do business with them because of their commitment to index arbitrage , officials inside and outside these firms confirmed ."
#%% md
### constituent
#%%
def constituent_parsing(sent):
    return nlp.parse(sentence).replace('ROOT', " ").replace('\n', '').replace(' ', '')
#%%
# constituent_parsing(sentence)
#%% md
### dependency
#%%
def dependency_parse(sent):
    s = tokenizor(sent)
    tokens = [token.text for token in s]
    tokens.insert(0,'ROOT')
    dependency = []
    for tag, start, end in nlp.dependency_parse(sent):
        dependency.append([tag, f'{tokens[start]}-{start}', f'{tokens[end]}-{end}'])
    return dependency
#%%
# dependency_parse(sentence)
#%% md
## preparing pdtb --testing on trial
#%%
def pdtb_to_json(folder, name):
    p = Path(folder)
    with open(p/name) as f:
        file = f.read()
        doc = tokenizor(file)

        sentences = []
        for idno, sent in enumerate(doc.sents):
            sent = str(sent)
            if idno == 0:
                continue
            dependencies = dependency_parse(sent)
            parsetree = constituent_parsing(sent)

            words = []
            sent_index = file.index(sent)
            for token in tokenizor(sent):
                begin = sent_index+token.idx
                token_dict = {'CharacterOffsetBegin':begin, "CharacterOffsetEnd": begin+len(token)}
                words.append([token.text, token_dict])
            sentences.append({"dependencies": dependencies, "parsetree": parsetree, "words": words})
    return json.dumps({name: {"sentences": sentences}})
#%%
jsonFile = pdtb_to_json('/home/pengfei/data/trial/raw_train/', 'wsj_1000')
#%%
with open('pdtb-parses.json', 'w') as f:
    json.dump(jsonFile, f)
#%%
with open('a.txt', 'w') as f:
    f.write('a')
#%%